<a href="https://colab.research.google.com/github/rtyx/udacity_pytorch_challenge/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

!pip install pillow==4.1.1
%reload_ext autoreload
%autoreload

In [0]:
# Imports here

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from collections import OrderedDict
from torch.autograd import Variable
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
import time
import datetime
import os
import copy
from matplotlib.ticker import FormatStrFormatter


plt.ion()   # interactive mode

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [4]:
current_work_directory = os.getcwd()
abs_work_directory = os.path.abspath(current_work_directory)

data_dir = "/content/gdrive/My Drive/udacity_pytorch_challenge" + '/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

print('Train directory: {}'.format(train_dir))
print('Validation directory: {}'.format(valid_dir))
print('Test directory: {}'.format(valid_dir))


Train directory: /content/gdrive/My Drive/udacity_pytorch_challenge/flower_data/train
Validation directory: /content/gdrive/My Drive/udacity_pytorch_challenge/flower_data/valid
Test directory: /content/gdrive/My Drive/udacity_pytorch_challenge/flower_data/valid


In [0]:
# TODO: Define your transforms for the training and validation sets
IMG_CROP = 224
IMG_RESIZE = 256
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
ROTATION = 4
JITTER = .3

train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(IMG_CROP),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(ROTATION),
        transforms.ColorJitter(JITTER, JITTER, JITTER),
        transforms.ToTensor(),
        transforms.Normalize(mean=MEAN, std=STD)
     ])
validation_transforms = transforms.Compose([
        transforms.Resize(IMG_RESIZE),
        transforms.CenterCrop(IMG_CROP),
        transforms.ToTensor(),
        transforms.Normalize(mean=MEAN, std=STD)
     ])
test_transforms = transforms.Compose([
        transforms.Resize(IMG_RESIZE),
        transforms.CenterCrop(IMG_CROP),
        transforms.ToTensor(),
        transforms.Normalize(mean=MEAN, std=STD)
     ])

data_transforms = {
    'train': train_transforms,
    'validation': validation_transforms,
    'test': test_transforms
}

train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
validation_data = datasets.ImageFolder(valid_dir, transform=validation_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

image_datasets = {
    'train': train_data,
    'validation': validation_data,
    'test': test_data
}

dataset_sizes = {x: len(image_datasets[x]) 
                              for x in ['train', 'validation', 'test']}

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size =32,shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size =32,shuffle=False)

data_loaders = {
    'train': train_loader,
    'validation': validation_loader,
    'test': test_loader
}

class_names = image_datasets['train'].classes

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [0]:
import json

with open("/content/gdrive/My Drive/udacity_pytorch_challenge" + '/cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [0]:
# TODO: Build and train your network
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [0]:
def train_model(model, criteria, optimizer, scheduler, num_epochs=8, device=device):
    print("Training model...")
    
    model.to(device)
  
    since = time.time()
    
    print(datetime.datetime.fromtimestamp(since).strftime('%Y-%m-%d %H:%M:%S'))

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                scheduler.step()
                model.train()
                print("Training...")
            else:
                model.eval()
                print("Evaluating...")

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in data_loaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criteria(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Accuracy: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best Accuracy: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
model = models.vgg19(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

classifier = nn.Sequential(OrderedDict([
                     ('dr', nn.Dropout(p=0.2)),
                     ('fc1', nn.Linear(25088, 4096)),
                     ('relu', nn.ReLU()),
                     ('fc2', nn.Linear(4096, 1024)),
                     ('relu', nn.ReLU()),
                     ('fc3', nn.Linear(1024, 512)),
                     ('relu', nn.ReLU()),
                     ('fc4', nn.Linear(512, 102)),
                     ('output', nn.LogSoftmax(dim=1))
                     ]))

model.classifier = classifier

criteria = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

scheduler = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

epochs=12

In [0]:
my_model = train_model(model, criteria, optimizer, scheduler, epochs, device)


Training model...
2019-01-09 00:38:48
Epoch 1/12
----------
Training...
train Loss: 3.1899 Accuracy: 0.2921
Evaluating...
validation Loss: 1.5595 Accuracy: 0.5636

Epoch 2/12
----------
Training...
train Loss: 1.7057 Accuracy: 0.5459
Evaluating...
validation Loss: 1.1210 Accuracy: 0.7078

Epoch 3/12
----------
Training...
train Loss: 1.4320 Accuracy: 0.6113
Evaluating...
validation Loss: 0.9157 Accuracy: 0.7604

Epoch 4/12
----------
Training...


In [0]:
def calc_accuracy(model, data, cuda=False):
    model.eval()
    model.to(device)    
    
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(data_loaders[data]):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model.forward(inputs)
            _, predicted = outputs.max(dim=1)
            if idx == 0:
                print(predicted)
                print(torch.exp(_))
            equals = predicted == labels.data
            if idx == 0:
                print(equals)
            print(equals.float().mean())

In [0]:
calc_accuracy(model, 'test', True)

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [0]:
# TODO: Save the checkpoint 
model.class_to_idx = train_data.class_to_idx
model.cpu()
checkpoint = {'state_dict': model.state_dict(),
              'class_to_idx': model.class_to_idx}
torch.save(checkpoint, "/content/gdrive/My Drive/udacity_pytorch_challenge/checkpoint_23_12.pth")

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [0]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=lambda storage, loc: storage)
    model = models.vgg19(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    classifier = nn.Sequential(OrderedDict([
                     ('dr', nn.Dropout(p=0.2)),
                     ('fc1', nn.Linear(25088, 4096)),
                     ('relu', nn.ReLU()),
                     ('fc2', nn.Linear(4096, 1024)),
                     ('relu', nn.ReLU()),
                     ('fc3', nn.Linear(1024, 512)),
                     ('relu', nn.ReLU()),
                     ('fc4', nn.Linear(512, 102)),
                     ('output', nn.LogSoftmax(dim=1))
                     ]))

    model.classifier = classifier
    criteria = nn.NLLLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
    model.load_state_dict(checkpoint['state_dict'], strict=False)
    model.eval()
    return model
    
    
load_checkpoint("/content/gdrive/My Drive/udacity_pytorch_challenge/checkpoint_23_12.pth")  
print(model)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [0]:
def process_image(image):
    img_pil = Image.open(image)
   
    adjustments = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    img_tensor = adjustments(img_pil)
    
    return img_tensor

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [0]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    image = image.numpy().transpose((1, 2, 0))
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

imshow(process_image(data_dir + '/test' + '/1/' + 'image_06773.jpg'))

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [0]:
model.class_to_idx =train_data.class_to_idx

ctx = model.class_to_idx


def predict(image_path, model, topk=5):   
    model.to(device)
    img_torch = process_image(image_path)
    img_torch = img_torch.unsqueeze_(0)
    img_torch = img_torch.float()
    
    with torch.no_grad():
        output = model.forward(img_torch.to(device))
        
    probability = F.softmax(output.data,dim=1)
    
    return probability.topk(topk)

In [0]:
img = (data_dir + '/test' + '/1/' + 'image_06773.jpg')

val1, val2 = predict(img, model)
print(val1)
print(val2)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [0]:
# TODO: Display an image along with the top 5 classes
def check_sanity(folder, imageName):
    plt.rcParams["figure.figsize"] = (10,5)
    plt.subplot(211)
    
    index = folder
    path = data_dir + '/test' + '/' + folder + '/' + imageName

    probabilities = predict(path, model)
    image = process_image(path)
    probabilities = probabilities
    

    axs = imshow(image, ax = plt)
    axs.axis('off')
    axs.title(cat_to_name[str(index)])
    axs.show()
    
    
    a = np.array(probabilities[0][0])
    b = [cat_to_name[str(index + 1)] for index in np.array(probabilities[1][0])]
    
    
    N=float(len(b))
    fig,ax = plt.subplots(figsize=(8,3))
    width = 0.8
    tickLocations = np.arange(N)
    ax.bar(tickLocations, a, width, linewidth=4.0, align = 'center')
    ax.set_xticks(ticks = tickLocations)
    ax.set_xticklabels(b)
    ax.set_xlim(min(tickLocations)-0.6,max(tickLocations)+0.6)
    ax.set_yticks([0.2,0.4,0.6,0.8,1,1.2])
    ax.set_ylim((0,1))
    ax.yaxis.grid(True)
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

    plt.show()

In [0]:
check_sanity('1', 'image_06772.jpg')
check_sanity('1', 'image_06773.jpg')
check_sanity('2', 'image_05145.jpg')
check_sanity('2', 'image_05146.jpg')
check_sanity('6', 'image_08111.jpg')
check_sanity('6', 'image_08110.jpg')
